In [1]:
import sys
sys.path.append("/home/user/diplom/tvm/python")

In [2]:
import tvm
from tvm import relax
import numpy as np

In [4]:
from torchvision import transforms

test_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

/home/user/diplom/tvm-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
fer_model_name = "/home/user/diplom/fer.so"
vm_fer = relax.VirtualMachine(tvm.runtime.load_module(fer_model_name), tvm.cpu())

In [47]:
import tvm
import numpy as np
import pandas as pd

import os

from tvm.script.parser import ir as I
from tvm.script import relax as R

from tvm.relax.training import SetupTrainer
from tvm.relax.training.trainer import Trainer
from tvm.relax.training.optimizer import Adam
from tvm.relax.training.loss import CategoricalCrossEntropyLoss

In [37]:
@I.ir_module
class FER:
    I.module_attrs({"param_num": 2, "state_num": 0})
    
    @R.function
    def backbone(
        x: R.Tensor((1, 1280), "float32"),
        w0: R.Tensor((1280, 8), "float32"),
        b0: R.Tensor((1, 8), "float32"),
    ):
        with R.dataflow():
            lv0 = R.matmul(x, w0)
            lv1 = R.add(lv0, b0)
            out = R.nn.softmax(lv1, axis = 1)
            R.output(out)
        return out
    
emotions = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']

In [38]:
pred_sinfo = relax.TensorStructInfo((1, 8), "float32")
target_sinfo = relax.TensorStructInfo((1, 8), "int32")
setup_trainer = SetupTrainer(
    CategoricalCrossEntropyLoss(reduction="mean"),
    Adam(0.0001),
    [pred_sinfo, target_sinfo],
)

train_mod = setup_trainer(FER)

dev = tvm.device("cpu", 0)
target="cpu"

trainer = Trainer(train_mod, vm_fer, dev, False)

In [39]:
import pickle

with open('/home/user/diplom/fer_model_params.pkl', 'rb') as f:
    model_params = pickle.load(f)
trainer.load_params(model_params)

In [40]:
def load_features(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)

In [41]:
features = load_features('/home/user/diplom/all_features/features05-07.pickle')

In [29]:
target_type = "int32"
label = np.array([[0, 0, 1, 0, 0, 0, 0, 0]]).astype(target_type)

In [43]:
trainer.update(features[0], label)

<tvm.nd.NDArray shape=(), cpu(0)>
array(0.16105825, dtype=float32)

In [ ]:
last_loss = np.inf
for epoch in range(5):
    loss = 0
    batch_size = 60000
    for i in range(batch_size):
        loss += trainer.update_params(features[i], dataset[i][1]).numpy()
    print("#epoch", epoch, "loss=", loss / batch_size)

In [44]:
trainer.predict(features[40])

<tvm.nd.NDArray shape=(1, 8), cpu(0)>
array([[1.3907691e-16, 1.7230758e-13, 9.9957722e-01, 6.4557399e-19,
        4.0041466e-04, 3.1970312e-11, 7.5862937e-07, 2.1688054e-05]],
      dtype=float32)

### split

In [46]:
! ls all_features

features01-03.pickle  features04-03.pickle  features06-01.pickle
features01-08.pickle  features04-04.pickle  features06-08.pickle
features01-11.pickle  features04-05.pickle  features06-13.pickle
features01-13.pickle  features04-09.pickle  features07-03.pickle
features02-06.pickle  features04-14.pickle  features07-06.pickle
features02-08.pickle  features05-05.pickle  features07-07.pickle
features02-14.pickle  features05-06.pickle  features08-01.pickle
features03-03.pickle  features05-07.pickle  features08-05.pickle
features03-07.pickle  features05-08.pickle  features08-08.pickle
features03-14.pickle  features05-09.pickle  features08-12.pickle


In [48]:
DATA_DIR = 'all_features'
all_features = os.listdir(DATA_DIR)

In [58]:
def get_label(features_file):
    return int(features_file.split('-')[0][-2:])

In [57]:
all_labels = [0] * 8
for features_file in all_features:
    features = load_features()
    all_labels[]


4
2
1
4
3
4
0
5
6
3
2
7
4
7
3
5
1
2
3
0
7
0
6
5
7
6
4
3
0
1
